# Step 1: Data Acquisition

In [26]:
import pandas as pd

**Note: The following cells are the professor's work unless specific lines are denoted otherwise**

In [27]:
# 
# These are standard python modules
import json, time, urllib.parse
#
# The 'requests' module is not a standard Python module. You will need to install this with pip/pip3 if you do not already have it
import requests

LO: Added my UW email below, changed start and end dates

In [28]:
#########
#
#    CONSTANTS
#

# The REST API 'pageviews' URL - this is the common URL/endpoint for all 'pageviews' API requests
API_REQUEST_PAGEVIEWS_ENDPOINT = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/'

# This is a parameterized string that specifies what kind of pageviews request we are going to make
# In this case it will be a 'per-article' based request. The string is a format string so that we can
# replace each parameter with an appropriate value before making the request
API_REQUEST_PER_ARTICLE_PARAMS = 'per-article/{project}/{access}/{agent}/{article}/{granularity}/{start}/{end}'

# The Pageviews API asks that we not exceed 100 requests per second, we add a small delay to each request
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making a request to the Wikimedia API they ask that you include your email address which will allow them
# to contact you if something happens - such as - your code exceeding rate limits - or some other error 
REQUEST_HEADERS = {
    'User-Agent': '<obrienl@uw.edu>, University of Washington, MSDS DATA 512 - AUTUMN 2023',
}

# This is just a list of English Wikipedia article titles that we can use for example requests
ARTICLE_TITLES = [ 'Bison', 'Northern flicker', 'Red squirrel', 'Chinook salmon', 'Horseshoe bat' ]

# This template is used to map parameter values into the API_REQUST_PER_ARTICLE_PARAMS portion of an API request. The dictionary has a
# field/key for each of the required parameters. In the example, below, we only vary the article name, so the majority of the fields
# can stay constant for each request. Of course, these values *could* be changed if necessary.
ARTICLE_PAGEVIEWS_PARAMS_TEMPLATE = {
    "project":     "en.wikipedia.org",
    "access":      "desktop",      # this should be changed for the different access types
    "agent":       "user",
    "article":     "",             # this value will be set/changed before each request
    "granularity": "monthly",
    "start":       "2015070100",   # start and end dates need to be set
    "end":         "2023093000"    # this is likely the wrong end date
}


The API request will be made using one procedure. The idea is to make this reusable. The procedure is parameterized, but relies on the constants above for the important parameters. The underlying assumption is that this will be used to request data for a set of article pages. Therefore the parameter most likely to change is the article_title.

In [29]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_pageviews_per_article(article_title = None, 
                                  endpoint_url = API_REQUEST_PAGEVIEWS_ENDPOINT, 
                                  endpoint_params = API_REQUEST_PER_ARTICLE_PARAMS, 
                                  request_template = ARTICLE_PAGEVIEWS_PARAMS_TEMPLATE,
                                  headers = REQUEST_HEADERS):

    # article title can be as a parameter to the call or in the request_template
    if article_title:
        request_template['article'] = article_title

    if not request_template['article']:
        raise Exception("Must supply an article title to make a pageviews request.")

    # Titles are supposed to have spaces replaced with "_" and be URL encoded
    article_title_encoded = urllib.parse.quote(request_template['article'].replace(' ','_'))
    request_template['article'] = article_title_encoded
    
    # now, create a request URL by combining the endpoint_url with the parameters for the request
    request_url = endpoint_url+endpoint_params.format(**request_template)
    
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like Wikipedia - or other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(request_url, headers=headers)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response


In [9]:
.json?

**end of direct copy of Prof's example**

## Pull Monthly Desktop Access

Exploring structure of views object

In [30]:
# Get the desktop data
views = request_pageviews_per_article('Inside_Out_(2015_film)')

In [31]:
views

{'items': [{'project': 'en.wikipedia',
   'article': 'Inside_Out_(2015_film)',
   'granularity': 'monthly',
   'timestamp': '2015070100',
   'access': 'desktop',
   'agent': 'user',
   'views': 610075},
  {'project': 'en.wikipedia',
   'article': 'Inside_Out_(2015_film)',
   'granularity': 'monthly',
   'timestamp': '2015080100',
   'access': 'desktop',
   'agent': 'user',
   'views': 375046},
  {'project': 'en.wikipedia',
   'article': 'Inside_Out_(2015_film)',
   'granularity': 'monthly',
   'timestamp': '2015090100',
   'access': 'desktop',
   'agent': 'user',
   'views': 289573},
  {'project': 'en.wikipedia',
   'article': 'Inside_Out_(2015_film)',
   'granularity': 'monthly',
   'timestamp': '2015100100',
   'access': 'desktop',
   'agent': 'user',
   'views': 301467},
  {'project': 'en.wikipedia',
   'article': 'Inside_Out_(2015_film)',
   'granularity': 'monthly',
   'timestamp': '2015110100',
   'access': 'desktop',
   'agent': 'user',
   'views': 289496},
  {'project': 'en.wik

In [12]:
type(views)

dict

In [13]:
views['items']

[{'project': 'en.wikipedia',
  'article': 'Everything_Everywhere_All_at_Once',
  'granularity': 'monthly',
  'timestamp': '2020010100',
  'access': 'desktop',
  'agent': 'user',
  'views': 1209},
 {'project': 'en.wikipedia',
  'article': 'Everything_Everywhere_All_at_Once',
  'granularity': 'monthly',
  'timestamp': '2020020100',
  'access': 'desktop',
  'agent': 'user',
  'views': 2944},
 {'project': 'en.wikipedia',
  'article': 'Everything_Everywhere_All_at_Once',
  'granularity': 'monthly',
  'timestamp': '2020030100',
  'access': 'desktop',
  'agent': 'user',
  'views': 2612},
 {'project': 'en.wikipedia',
  'article': 'Everything_Everywhere_All_at_Once',
  'granularity': 'monthly',
  'timestamp': '2020040100',
  'access': 'desktop',
  'agent': 'user',
  'views': 4530},
 {'project': 'en.wikipedia',
  'article': 'Everything_Everywhere_All_at_Once',
  'granularity': 'monthly',
  'timestamp': '2020050100',
  'access': 'desktop',
  'agent': 'user',
  'views': 3952},
 {'project': 'en.wik

In [14]:
type(views['items'])

list

Thoughts: views is a dictionary containing a key called items and a value which is a list of months of data

In [15]:
print(json.dumps(views,indent=4))

{
    "items": [
        {
            "project": "en.wikipedia",
            "article": "Everything_Everywhere_All_at_Once",
            "granularity": "monthly",
            "timestamp": "2020010100",
            "access": "desktop",
            "agent": "user",
            "views": 1209
        },
        {
            "project": "en.wikipedia",
            "article": "Everything_Everywhere_All_at_Once",
            "granularity": "monthly",
            "timestamp": "2020020100",
            "access": "desktop",
            "agent": "user",
            "views": 2944
        },
        {
            "project": "en.wikipedia",
            "article": "Everything_Everywhere_All_at_Once",
            "granularity": "monthly",
            "timestamp": "2020030100",
            "access": "desktop",
            "agent": "user",
            "views": 2612
        },
        {
            "project": "en.wikipedia",
            "article": "Everything_Everywhere_All_at_Once",
            "granul

To do:
- change "items" to name of current article
- remove "access" field
- append to JSON file

**Make list of article names:**

https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html

In [23]:
df_articles = pd.read_excel('thank_the_academy.AUG.2023.csv.xlsx')
df_articles.describe()

,name,url
count,1359,1359
unique,1359,1359
top,Everything Everywhere All at Once,https://en.wikipedia.org/wiki/Everything_Every...
freq,1,1


In [25]:
article_names = df_articles['name']

In [16]:
# title_list = ['Everything Everywhere All at Once', 'The Whale (2022 film)']

References:
- https://www.geeksforgeeks.org/append-to-json-file-using-python/#

In [19]:
# num_articles = 0
articles_data = {}

# For each article
for title in title_list:
    # Get data from API for article
    article_views = request_pageviews_per_article(title)
    
    month_list = []
    
    # Modify the retrieved data -------------!!!!!!!!!!!! Got idea for this form of iterating from Prof example
    for month in article_views['items']:
        # Remove access key from each month's entry
        del month['access'] # learned from https://www.geeksforgeeks.org/python-ways-to-remove-a-key-from-dictionary/
        
        # Reconstruct list of months
        month_list.append(month)
        
        # Store in a dictionary of articles
        articles_data[title] = month_list
        
# Print results
# print(json.dumps(articles_data, indent=4)) # !!!! line idea from prof example

# Save to JSON file - Reference: https://www.geeksforgeeks.org/writing-to-file-in-python/
my_file = open(r"academy_monthly_desktop_201507-202309.json", "w")
my_file.write(json.dumps(articles_data, indent=4))
my_file.close()
        
    
    #Append data to JSON file
#     if(num_articles == 0):
#        articles_data = 
#     else:
#         article_data.update(article_views)
#     article_data.update(article_views)
    
    # Save JSON File
    
        
        
    

**Make list of article names:**

https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html

In [23]:
df_articles = pd.read_excel('thank_the_academy.AUG.2023.csv.xlsx')
df_articles.describe()

,name,url
count,1359,1359
unique,1359,1359
top,Everything Everywhere All at Once,https://en.wikipedia.org/wiki/Everything_Every...
freq,1,1


In [25]:
article_names = df_articles['']